In [ ]:
import warnings
from pathlib import Path

import iris
import numpy as np

In [ ]:
from aeolus.coord import coord_to_cube, regrid_3d, replace_z_coord, volume_weights_cube
from aeolus.model import um

In [ ]:
from util_commons import STASH, SUITES

In [ ]:
warnings.filterwarnings("ignore", module="iris")

In [ ]:
# Select planet, experiment and metallicity
planet = "hd209458b"
exp = "kinetics"
metallicity = "1x solar"

if planet == "hatp11b" and exp == "kinetics" and metallicity == "100solar":
    npath_1 = SUITES[planet][exp][metallicity]["dir_for_raw_1"]
    npath_2 = SUITES[planet][exp][metallicity]["dir_for_raw_2"]
    flist_1 = []
    flist_2 = []
    for file in npath_1.glob("**/atmosa.p*"):  # this directory and all subdirectories, recursively
        # Exclude the file containing 1 time slice for day 1000; use 5 time slices for days 1000-1050 instead
        if "/bd257/um_runs/kinetics/hatp11_rt/u-bl344/raw_output/atmos_base_1010/" not in str(file):
            if not str(file).endswith("arch"):
                if (
                    "atmosa.pb" in file.name
                    or "atmosa.pc" in file.name
                    or "atmosa.pd" in file.name
                    or "atmosa.pe" in file.name
                    or "atmosa.pf" in file.name
                    or "atmosa.pg" in file.name
                ):
                    flist_1.append(str(file))
    for file in npath_2.glob("**/atmosa.p*"):
        if not str(file).endswith("arch"):
            if (
                "atmosa.pb" in file.name
                or "atmosa.pc" in file.name
                or "atmosa.pd" in file.name
                or "atmosa.pe" in file.name
                or "atmosa.pf" in file.name
                or "atmosa.pg" in file.name
            ):
                flist_2.append(str(file))
    flist = flist_1 + flist_2
else:
    npath = SUITES[planet][exp][metallicity]["dir_for_raw"]
    flist = []
    for file in npath.glob("**/atmosa.p*"):
        if (
            "atmosa.pb" in file.name
            or "atmosa.pc" in file.name
            or "atmosa.pd" in file.name
            or "atmosa.pe" in file.name
            or "atmosa.pf" in file.name
            or "atmosa.pg" in file.name
        ):
            flist.append(str(file))
# Print sorted list of raw files
# len(sorted(flist))

In [ ]:
# Load raw data
cl = iris.load(flist, STASH.keys())  # 38 fields

In [ ]:
# Create a folder for storing merged data
(SUITES[planet][exp][metallicity]["dir_for_merged"]).mkdir(parents=False, exist_ok=True)
if planet == "hatp11b" and exp == "equilibrium" and metallicity == "100solar":
    fname = f"{SUITES[planet][exp][metallicity]['rose_suite']}_100timesSolar.nc"
elif planet == "hatp11b" and exp == "kinetics" and metallicity == "100solar":
    fname = f"{SUITES[planet][exp][metallicity]['rose_suite']}_100timesSolar.nc"
else:
    fname = f"{SUITES[planet][exp][metallicity]['rose_suite']}.nc"

In [ ]:
# Rename chemical species from STASH items to "<specie> mole fraction"
for key, value in STASH.items():
    for cube in cl:
        if key == cube.name() and "mole fraction" in value:
            cube.rename(value)
            cube.units = "1"

In [ ]:
# Convert PosixPaths to strings to be able to save them to .nc as metadata
for key, value in SUITES[planet][exp][metallicity].items():
    if "dir" in key:
        SUITES[planet][exp][metallicity][key] = str(value)

cl_on_hlevs = iris.cube.CubeList()
for cube in cl:
    # Remove the lowest model_level_number to match other variables
    if cube.name() in ["air_pressure", "upward_air_velocity"]:
        new_cube = cube[:, 1:, ...]
    else:
        new_cube = cube
    # Replace model_level_number by level_height as a dimensional vertical coordinate
    if "model_level_number" in [coord.name() for coord in new_cube.dim_coords]:
        new_cube = replace_z_coord(new_cube)
    # Add metadata
    new_cube.attributes.update(SUITES[planet][exp][metallicity])
    cl_on_hlevs.append(new_cube)

In [ ]:
# Save to .nc
iris.save(
    cl_on_hlevs,
    str(Path(SUITES[planet][exp][metallicity]["dir_for_merged"]) / fname),
)